In [4]:
# we will see how to stack decorators.

def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone

    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f'{run_dt}: called {fn.__name__}')
        return result

    return inner


def timed(fn):
    from time import perf_counter # so we can reuse the decorator somewhere else
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs) # this makes the closure since fn is local to timed and nonlocal to inner.
        end = perf_counter()
        elapsed = end - start

        args_ = [str(i) for i in args]
        kwargs_ = [f'{k}={v}' for k,v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)

        print(f"{fn.__name__}({args_str}) took {elapsed: .6f}s to run")
        return result
    return inner # return our closure.


@logged
def func_1():
    pass

func_1()

2022-02-01 22:28:38.258886+00:00: called func_1


In [9]:
@logged
@timed
def fact(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

# The same as having
def fact_2(n):
    from operator import mul
    from functools import reduce

    return reduce(mul, range(1, n+1))

fact_2 = logged(timed(fact_2))

'''
It is important to note that the first function that runs is logged that gets passed timed as a parameter, the fact that we 
first run the function inside logged and after we print the output means that we see first what happens in timed, however,
it was logged called with timed as a parameter which in turn called fact_2 as a parameter.
'''

In [11]:
fact(3)

fact(3) took  0.000010s to run
2022-02-01 22:29:58.538602+00:00: called fact


6

In [10]:
fact_2(3)

fact_2(3) took  0.000009s to run
2022-02-01 22:29:56.733922+00:00: called fact_2


6

In [16]:
# it first runs print and THEN runs and returns the function.
def dec_1(fn):
    def inner():
        print('Running Dec_1')
        return fn()
    return inner

def dec_2(fn):
    def inner():
        print('Running Dec_2')
        return fn()
    return inner

@dec_1
@dec_2
def my_func():
    print("I am running")

# my_func = dec_1(dec_2(my_func))

my_func()

Running Dec_1
Running Dec_2
I am running


In [15]:
# it first runs the function THEN prints and returns
def dec_1(fn):
    def inner():
        result = fn()
        print('Running Dec_1')
        return result
    return inner

def dec_2(fn):
    def inner():
        result = fn()
        print('Running Dec_2')
        return result
    return inner

@dec_1
@dec_2
def my_func():
    print("I am running")

# my_func = dec_1(dec_2(my_func))

my_func()

I am running
Running Dec_2
Running Dec_1
